This notebook shows how to compare 2D datasets from ROOT and McStas simulations 

- load McStas and ROOT data
- crop ROOT data 
- resample x-y grid to the smallest grid 
- normalize intensity
- choose centering points
- plot difference as 2D and 3D plots

**Note**:  The physical units were not considered when comparing the arrays.

In [ ]:
import os
import pprint
import uproot
import matplotlib
import matplotlib.pyplot as plt
import dataconfig  # to get paths to data
%matplotlib widget

import scipy.interpolate

import numpy as np

import ipywidgets

colormap = plt.cm.get_cmap('gist_earth')
# other colormaps" 'viridis', 'seismic'

matplotlib.rcParams['figure.max_open_warning'] = 0

# Load McStas data

In [ ]:
# Output of McStas simulation
selected_filename = 'monitor_tx_DENEX.dat'

path_to_mcstas2D_file = os.path.join(dataconfig.data_mcstas, selected_filename)

assert os.path.isdir(dataconfig.data_mcstas), \
'The folder which should contain outputs of McStas simulation does not exist.'

assert os.path.isfile(path_to_mcstas2D_file), \
    'There is an issue with the chosen McStas 2D datafile'

In [ ]:
# Create a dictionary containing filenames, shape of output data, position of component and x, y labels
dict_selected_mcstas_file = {}

with open(path_to_mcstas2D_file, 'r') as file:
    for line in file:
        if 'xlabel' in line:
            dict_selected_mcstas_file['xlabel'] = ' '.join(line.rstrip().split()[-2:])
        elif 'ylabel' in line:
            dict_selected_mcstas_file['ylabel'] = ' '.join(line.rstrip().split()[-2:])
        elif 'position' in line:
            dict_selected_mcstas_file['position'] = line.rstrip().split()[-3:]
        elif "array_2d" in line:
            type_array = line.rstrip()
            start = type_array.find('(') + 1
            end = type_array.find(')', start)
            dict_selected_mcstas_file['nx_value'], dict_selected_mcstas_file['ny_value'] = type_array[start:end].split(',')

print(f'Information about plotted McStas 2D data {selected_filename}')
pprint.pprint(dict_selected_mcstas_file)

In [ ]:
data2d = np.genfromtxt(path_to_mcstas2D_file,
                       max_rows=int(dict_selected_mcstas_file['ny_value']))

#if position_x of component < 0, flip data along y axis 
# otherwise leave unchanged
if float(dict_selected_mcstas_file['position'][0]) < 0:
     data2d = np.flip(data2d, 0)

fig1, ax1 = plt.subplots(constrained_layout=True)
CS = ax1.contourf(data2d, cmap=colormap)    
ax1.set_title(selected_filename)
cbar = fig1.colorbar(CS)
fig1.canvas.header_visible = False
fig1.canvas.toolbar_position = 'bottom'

# Load ROOT data

In [ ]:
path_to_root_file = dataconfig.data_root  

assert os.path.isdir(path_to_root_file), \
    'The path to the folder which should contain ROOT files does not exist.'

In [ ]:
# open one ROOT file and extract only one 1D and one 2D dataset specified in dict_selected_dataset
# The 1D and 2D datasets are stored in a dictionary
# Note the vertical axis of 2D datasets is inverted

key_spectrum ='Spectrum03'
ROOTfile = 'Spectrum03_DENEX006_1_18-02-05_0000.root'
dir_with_data = 'Meas_3'
selected_dataset = 'H_TOF,X1-X2_User_2D2_dsp_after_run_3'

file_to_open = os.path.join(path_to_root_file, ROOTfile)

assert os.path.isfile(file_to_open), 'There is an issue with the chosen ROOT file'

# dictionary to store the x, y boundaries of selected 2D root data
root_x_y_boundaries = {}

with uproot.open(file_to_open)[dir_with_data] as myFile:
    for key in myFile.keys():

        # 2D contourplot
        if 'TH2' in str(myFile[key]) and selected_dataset in str(key):
            key_name = myFile[key].name.decode('utf-8') 
            data2d_root =  np.flip(myFile[key].values, 1).transpose()
            
            # extract info about x, y axis (min, max and number of bins) 
            root_x_y_boundaries['x_min'] = myFile[key].xlow
            root_x_y_boundaries['x_max'] = myFile[key].xhigh 
            root_x_y_boundaries['y_min'] = myFile[key].ylow 
            root_x_y_boundaries['y_max'] = myFile[key].yhigh 
            
            x_min = root_x_y_boundaries['x_min']
            x_max = root_x_y_boundaries['x_max'] 
            bins_x = myFile[key].xnumbins 
            y_min = root_x_y_boundaries['y_min']
            y_max = root_x_y_boundaries['y_max']
            bins_y = myFile[key].ynumbins
            
            # create x- and y-axis
            xaxis = x_min + (x_max - x_min) / (bins_x - 1) * np.arange(bins_x)
            yaxis = y_min + (y_max - y_min) / (bins_y - 1) * np.arange(bins_y)
            
            name_output_file = ROOTfile[:10] + "_" + key_name.replace(',','_') + "_inv_y"

## Crop ROOT array (top, bottom)
Remove top and bottom rows of the ROOT array.
The boundaries can be set by the user using the slider.

In [ ]:
# Define function to plot when scaling is updated
def fct_scaling(set_limits):
    fig2, ax2 = plt.subplots(constrained_layout=True)
    # plot      
    CS = ax2.contourf(xaxis, yaxis, data2d_root, cmap=colormap)
    cbar = fig2.colorbar(CS)
    ax2.plot(xaxis, 0*xaxis+ yaxis[set_limits[0]], 'orange')
    ax2.plot(xaxis, 0*xaxis+ yaxis[set_limits[1]], 'orange')
    fig2.canvas.header_visible = False
    fig2.canvas.toolbar_position = 'bottom'

# set textboxes for the scaling formulas    
style_textbox = {'description_width': 'initial'}
default_value_ymin = 50
default_value_ymax = len(yaxis)-1-50

set_limits = ipywidgets.IntRangeSlider(
    value=[default_value_ymin, default_value_ymax],
    min=0,
    max=len(yaxis)-1,
    step=1,
    description='Index boundaries:',
    disabled=False,
    continuous_update=False,
    readout_format='d',
    style=style_textbox,
    description_tooltip="Select indices to crop array"
)

# Display plots and widgets
interactive_plot = ipywidgets.interactive_output(fct_scaling, {'set_limits': set_limits}) 

set_limits_widgets = ipywidgets.VBox([ipywidgets.Label('Use the slider to select the lower and upper boundaries to crop the array '), 
                                      set_limits])
display(set_limits_widgets, interactive_plot)

In [ ]:
# crop array using the boundaries defined in the previous cell
cropped_array = data2d_root[set_limits.value[0]:set_limits.value[1], :]

In [ ]:
# plot to check new cropped array
fig3, ax3 = plt.subplots(constrained_layout=True)
CS = ax3.contourf(cropped_array, cmap=colormap)
ax3.set_title(f"CROPPED {key_name}")
cbar = fig3.colorbar(CS)
fig3.canvas.header_visible = False
fig3.canvas.toolbar_position = 'bottom'

# Rebin to the smallest grid
The array with the finest grid is resampled to the coarser grid.

In [ ]:
print(f"Shape of ROOT data: {cropped_array.shape}")
print(f"Shape of McStas data: {dict_selected_mcstas_file['nx_value']},{dict_selected_mcstas_file['ny_value']}")

In [ ]:
def rebin(matrix_to_rebin, new_shape, **dict_x_y_boundaries):
    '''
    Function to resample the array with the largest number of bins to a smaller grid
    
    Two cases have to be considered if large grid = multiple of small grid or not
    '''
    old_shape = matrix_to_rebin.shape
    # if old shape is multiple of new shape
    if all([(old_shape[index_dim] % new_shape[index_dim])==0 for index_dim in [0, 1]]):
        sh = new_shape[0], a.new_shape[0]//new_shape[0], new_shape[1], a.new_shape[1]//new_shape[1]
        return matrix_to_rebin.reshape(sh).mean(-1).mean(1)
    else:
        # case when resampling with no multiplicity between new and old dimensions
        # code adapted from https://stackoverflow.com/questions/34689519/how-to-coarser-the-2-d-array-data-resolution
       
        x_min = dict_x_y_boundaries['x_min']
        x_max = dict_x_y_boundaries['x_max']
        y_min = dict_x_y_boundaries['y_min']
        y_max = dict_x_y_boundaries['y_max']
        
        # old x, y axes 
        xgrid_old  = np.linspace(x_min, x_max, old_shape[1])
        ygrid_old  = np.linspace(y_min, y_max, old_shape[0])
        
        print('Old dimensions', len(xgrid_old), len(ygrid_old))

        # new x, y axes
        xgrid_new  = np.linspace(x_min, x_max, new_shape[0])
        ygrid_new  = np.linspace(y_min, y_max, new_shape[1])

        hfunc = scipy.interpolate.interp2d(xgrid_old, ygrid_old, matrix_to_rebin)

        reshaped_data = np.zeros(new_shape[0] * new_shape[1])

        t = 0

        for i in range(0, new_shape[1], 1):
            for j in range(0, new_shape[0], 1):
                reshaped_data[t] = hfunc(xgrid_new[j], ygrid_new[i]) 
                t+=1    
                
        return reshaped_data.reshape(new_shape[1], new_shape[0])

In [ ]:
new_mat = rebin(cropped_array, data2d.shape, **root_x_y_boundaries)

In [ ]:
fig4, ax4 = plt.subplots(constrained_layout=True)
ax4.set_title('Resampled ROOT 2D array to different grid')
fig4.canvas.header_visible = False
fig4.canvas.toolbar_position = 'bottom'
CS = plt.pcolormesh(new_mat, cmap=colormap)
cbar = fig4.colorbar(CS)

In [ ]:
print(f"Shape of reshaped data: {new_mat.shape}")

# Rescale matrices' intensities
In order to compare the 2 matrices, the intensity has to be rescaled to the same values.

In [ ]:
def rescale_intensities(array_to_rescale):
    '''
    Function to rescale intensity of `array_to_rescale` to a 256-value scale
    '''
    min_arr = np.min(array_to_rescale)
    max_arr = np.max(array_to_rescale)
    size_x, size_y = array_to_rescale.shape
    
    for i in range(size_x):
        for j in range(size_y):
            array_to_rescale[i,j] = 256. * (array_to_rescale[i,j]- min_arr) / (max_arr - min_arr)
            
    return array_to_rescale

In [ ]:
rescaled_root = rescale_intensities(new_mat)
rescaled_mcstas = rescale_intensities(data2d)

print(f"""Shape of modified ROOT matrix: {rescaled_root.shape}
Shape of modified McStas matrix: {rescaled_mcstas.shape}
Max intensity of modified ROOT matrix: {np.max(rescaled_mcstas)}
Max intensity of modified ROOT matrix: {np.max(rescaled_root)}""")

# Choice of point to align arrays

In [ ]:
layout_boundedinttext_box = ipywidgets.Layout(width='18%', 
                                              height='100%')
aspect_ratio_array = rescaled_root.shape[0]/rescaled_root.shape[1]

def plot_with_selected_center(centre_x_root, centre_y_root, centre_x_mcstas, centre_y_mcstas):
    fig5, axes5 = plt.subplots(nrows=1, ncols=2, tight_layout=True)
    fig5.canvas.header_visible = False
    fig5.canvas.toolbar_position = 'bottom'
    
    CS0 = axes5[0].pcolormesh(rescaled_root, 
                              cmap=colormap)
    axes5[0].set_title('ROOT')
    axes5[0].plot(centre_x_root, 
                  centre_y_root, 
                  'r+', markersize='12', markeredgewidth='2') 
    
    CS1 = axes5[1].pcolormesh(rescaled_mcstas, 
                              cmap=colormap)
    axes5[1].set_title('McStas')
    axes5[1].plot(centre_x_mcstas, 
                  centre_y_mcstas, 
                  'r+', markersize='12', markeredgewidth='2')
    
# Slider widgets
centre_x_root = ipywidgets.BoundedIntText( 
    value=0,
    min=0,
    max=len(xaxis)-1,
    step=1,
    description='ROOT x:',
    disabled=False,
    readout_format='d',
    style=style_textbox,
    layout=layout_boundedinttext_box
)

centre_y_root = ipywidgets.BoundedIntText( 
    value=0,
    min=0,
    max=len(yaxis)-1,
    step=1,
    description='ROOT y:',
    disabled=False,
    readout_format='d',
    style=style_textbox,
    layout=layout_boundedinttext_box
)

centre_x_mcstas = ipywidgets.BoundedIntText( 
    value=0,
    min=0,
    max=len(xaxis)-1,
    step=1,
    description='McStas x:',
    disabled=False,
    readout_format='d',
    style=style_textbox,
    layout=layout_boundedinttext_box
)

centre_y_mcstas = ipywidgets.BoundedIntText( 
    value=0,
    min=0,
    max=len(yaxis)-1,
    step=1,
    description='McStas y:',
    disabled=False,
    readout_format='d',
    style=style_textbox,
    layout=layout_boundedinttext_box
)

# Display plots and widgets
interactive_plot = ipywidgets.interactive_output(plot_with_selected_center, 
                                                 {'centre_x_root': centre_x_root,     
                                                  'centre_y_root': centre_y_root,
                                                  'centre_x_mcstas': centre_x_mcstas,
                                                  'centre_y_mcstas': centre_y_mcstas})

set_center = ipywidgets.HBox([centre_x_root, 
                              centre_y_root, 
                              centre_x_mcstas, 
                              centre_y_mcstas])

display(set_center, interactive_plot)

# Calculate difference of arrays

In [ ]:
# Define difference array
x0_root = centre_x_root.value
y0_root = centre_y_root.value

x0_mcstas = centre_x_mcstas.value
y0_mcstas = centre_y_mcstas.value

print(x0_root, y0_root, x0_mcstas, y0_mcstas)

offset_x = x0_root - x0_mcstas
offset_y = y0_root - y0_mcstas

In [ ]:
diff_arrays = np.empty((rescaled_root.shape[0] - np.absolute(offset_y),
                        rescaled_root.shape[1] - np.absolute(offset_x)))

for i in range(diff_arrays.shape[0]):
    for j in range(diff_arrays.shape[1]):
        diff_arrays[i, j] = rescaled_root[i + max(0, offset_y), j + max(0, offset_x)]\
        - rescaled_mcstas[i + min(0, offset_y), j + min(0, offset_x)]

In [ ]:
# info about difference array: size, min and max intensities
print(f"""Shape of difference array: {diff_arrays.shape}
Max and min intensities of difference array: {np.max(diff_arrays)}, {np.min(diff_arrays)}""")

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig6 = plt.figure(figsize=(8, 8))
fig6.canvas.header_visible = False
fig6.canvas.toolbar_position = 'bottom'
gs6 = matplotlib.gridspec.GridSpec(2, 2, height_ratios=[1, 1]) 

# Plot of ROOT rescaled array with selected centering point
ax6 = plt.subplot(gs6[0, 0])
CSroot = ax6.pcolormesh(rescaled_root, cmap=colormap)
ax6.set_title('ROOT')
ax6.plot(centre_x_root.value, 
         centre_y_root.value, 
         'r+', markersize='12', markeredgewidth='2') 

# Plot of McStas rescaled array with selected centering point
ax6 = plt.subplot(gs6[0, 1])   
CSmcstas = ax6.pcolormesh(rescaled_mcstas, cmap=colormap)
ax6.set_title('McStas')
ax6.plot(centre_x_mcstas.value, 
         centre_y_mcstas.value, 
         'r+', markersize='12', markeredgewidth='2')

cbax_mcstas = fig6.add_axes([0.905, .53, 0.01, 0.35]) 
cbar_mcstas = fig6.colorbar(CSmcstas, cax = cbax_mcstas) 

# Plot of difference array in 3D
ax6 = plt.subplot(gs6[1, 0], projection='3d')
xs = np.arange(diff_arrays.shape[1])
ys = np.arange(diff_arrays.shape[0])
xv, yv = np.meshgrid(xs, ys)
surf = ax6.plot_surface(xv, yv, diff_arrays, cmap=colormap, linewidth=0)

# Plot of difference array in 2D
ax6 = plt.subplot(gs6[1, 1]) 
CSdiff = ax6.contourf(xv, yv, diff_arrays, cmap=colormap)

cbax_diff = fig6.add_axes([0.905, 0.11, 0.01, 0.35]) 
cbar_diff = fig6.colorbar(CSdiff, cax = cbax_diff) 

In [ ]:
# # 3D plot of difference
fig7 = plt.figure(figsize=(8, 6))
fig7.canvas.header_visible = False
fig7.canvas.toolbar_position = 'bottom'

ax7 = fig7.add_subplot(111, projection='3d')
xs = np.arange(diff_arrays.shape[1])
ys = np.arange(diff_arrays.shape[0])
xv, yv = np.meshgrid(xs, ys)
ax7.set_xlabel('x')
ax7.set_ylabel('y')
surf = ax7.plot_surface(xv, yv, diff_arrays, 
                        cmap=colormap, 
                        linewidth=0)

fig7.colorbar(surf, ax=ax7, shrink=0.5);

In [ ]:
# 2D plot of difference
fig8, ax8 = plt.subplots()
fig8.canvas.header_visible = False
fig8.canvas.toolbar_position = 'bottom'
CS = ax8.contourf(xv, yv, diff_arrays, cmap=colormap)
cbar = fig8.colorbar(CS)